### Script for function to pull vitals data

This function loads chart events (specified by user-entered ID) into a dataframe, filters events by ventilataed patients, determines the last reading before extubation, takes the mean of this value for any duplicate readings (happens when multiple sensors used simultanrously) and returns the data as a column in the master dataframe

#### impot useful libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### user-defined variables that change every time

In [2]:
event = "height"
data_dir = "../data/feathered/"
export_dir = "../data/processed/"

datafile = data_dir+event
mean_export_file = export_dir+event
std_export_file = export_dir+"std_"+event
meanColumns = ['hadm_id',event]
stdColumns = ['hadm_id','std_'+event]

In [3]:
df = pd.read_feather(datafile)
df['valuenum']= pd.to_numeric(df['valuenum'])

In [4]:
# to convert fahrenheit to celcius: (F-32) x 5/9
#df['valuenum'] = np.where(df['itemid']==223761,(df['valuenum']-32)*(5/9),df['valuenum'])

In [5]:
# convert height in cm to inches- divide by 2.74
df['valuenum'] = np.where(df['itemid']==226730,(df['valuenum']/2.54),df['valuenum'])

In [6]:
df

,index,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,endtime,re_intub_class,time_on_vent
0,0,10074556,20846673,32640247,2126-02-12 01:43:00,2126-02-12 14:30:00,226707,70,70.000000,Inch,0,2126-02-13 10:51:00,0,19.683332
1,1,10004235,24181354,30276431,2196-02-24 14:39:00,2196-02-28 18:02:00,226707,72,72.000000,Inch,0,2196-02-27 16:28:00,0,71.600000
2,2,10004235,24181354,30276431,2196-02-24 14:39:00,2196-02-28 18:02:00,226730,183,72.047244,cm,0,2196-02-27 16:28:00,0,71.600000
3,3,10005348,25239799,31523640,2130-10-27 12:06:00,2130-10-27 16:58:00,226707,73,73.000000,Inch,0,2130-10-28 04:35:00,0,11.833333
4,4,10005348,25239799,31523640,2130-10-27 12:06:00,2130-10-27 16:58:00,226730,185,72.834646,cm,0,2130-10-28 04:35:00,0,11.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24893,24893,19054240,21472555,38651833,2164-08-09 18:57:00,2164-08-09 21:11:00,226730,175,68.897638,cm,0,2164-08-10 11:20:00,0,14.250000
24894,24894,19060629,24896717,37873734,2189-01-04 20:35:00,2189-01-31 11:24:00,226707,64,64.000000,Inch,0,2189-01-20 17:57:00,0,377.450000
24895,24895,19060629,24896717,37873734,2189-01-04 20:35:00,2189-01-31 11:24:00,226730,162.56,64.000000,cm,0,2189-01-20 17:57:00,0,377.450000
24896,24896,19062760,22605930,30132996,2197-03-17 16:56:00,2197-03-19 10:54:00,226707,62,62.000000,Inch,0,2197-03-17 23:17:00,0,6.283333


#### Convert times to pandas datetime object, find the time difference between each event and the time of extubation ('endtime') and find the last time an event occurred before extubation

In [7]:
df['endtime'] = pd.to_datetime(df['endtime'])
df['charttime'] = pd.to_datetime(df['charttime'])
df['time_diff']= df['endtime']-df['charttime']

In [8]:
df['timediffhour'] = df['time_diff'] / np.timedelta64(1, 'h') # convert time to hours
df_pre = df.copy()
df_pre = df[df['timediffhour']>0 & (df['timediffhour']<2)]

In [9]:
df_pre['min']=df_pre.groupby('hadm_id')['timediffhour'].transform('min') # find the last reading 
            # time before extubation occured and place in a new channel ('min')

<ipython-input-9-1f0533c03414>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pre['min']=df_pre.groupby('hadm_id')['timediffhour'].transform('min') # find the last reading


In [10]:
last_value_pre = df_pre[df_pre['timediffhour'] == df_pre['min']] 
            # select only rows where 'timediffhour' is the same as the minimum time ('min')

In [11]:
last_value_pre

,index,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning,endtime,re_intub_class,time_on_vent,time_diff,timediffhour,min
0,0,10074556,20846673,32640247,2126-02-12 01:43:00,2126-02-12 14:30:00,226707,70,70.000000,Inch,0,2126-02-13 10:51:00,0,19.683332,1 days 09:08:00,33.133333,33.133333
1,1,10004235,24181354,30276431,2196-02-24 14:39:00,2196-02-28 18:02:00,226707,72,72.000000,Inch,0,2196-02-27 16:28:00,0,71.600000,3 days 01:49:00,73.816667,73.816667
2,2,10004235,24181354,30276431,2196-02-24 14:39:00,2196-02-28 18:02:00,226730,183,72.047244,cm,0,2196-02-27 16:28:00,0,71.600000,3 days 01:49:00,73.816667,73.816667
3,3,10005348,25239799,31523640,2130-10-27 12:06:00,2130-10-27 16:58:00,226707,73,73.000000,Inch,0,2130-10-28 04:35:00,0,11.833333,0 days 16:29:00,16.483333,16.483333
4,4,10005348,25239799,31523640,2130-10-27 12:06:00,2130-10-27 16:58:00,226730,185,72.834646,cm,0,2130-10-28 04:35:00,0,11.833333,0 days 16:29:00,16.483333,16.483333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24893,24893,19054240,21472555,38651833,2164-08-09 18:57:00,2164-08-09 21:11:00,226730,175,68.897638,cm,0,2164-08-10 11:20:00,0,14.250000,0 days 16:23:00,16.383333,16.383333
24894,24894,19060629,24896717,37873734,2189-01-04 20:35:00,2189-01-31 11:24:00,226707,64,64.000000,Inch,0,2189-01-20 17:57:00,0,377.450000,15 days 21:22:00,381.366667,381.366667
24895,24895,19060629,24896717,37873734,2189-01-04 20:35:00,2189-01-31 11:24:00,226730,162.56,64.000000,cm,0,2189-01-20 17:57:00,0,377.450000,15 days 21:22:00,381.366667,381.366667
24896,24896,19062760,22605930,30132996,2197-03-17 16:56:00,2197-03-19 10:54:00,226707,62,62.000000,Inch,0,2197-03-17 23:17:00,0,6.283333,0 days 06:21:00,6.350000,6.350000


In [12]:
avg_dupValues = last_value_pre.groupby('hadm_id')['valuenum'].mean() # where duplicate values exist,take the mean
meanBP = avg_dupValues.reset_index()
meanBP.columns = (meanColumns)

In [13]:
svents = pd.read_csv(export_dir +'sample_vents.csv')
mean_df = pd.merge(left = svents, right=meanBP, how = 'left', left_on='hadm_id', right_on='hadm_id')   

In [20]:
mean_df

,hadm_id,endtime,time_on_vent,re_intub_class,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag,hours_to_death,height
0,28038802,2185-12-20 09:59:00,108.800000,0,18917458,2185-12-15 00:17:00,2185-12-22 17:15:00,NaN,EW EMER.,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,MARRIED,WHITE,2185-12-14 23:05:00,2185-12-15 01:31:00,0,NaN,NaN
1,21790335,2140-03-11 13:44:00,33.000000,0,19704964,2140-03-09 14:44:00,2140-03-15 13:00:00,NaN,URGENT,PHYSICIAN REFERRAL,HOME,Other,?,MARRIED,UNABLE TO OBTAIN,NaN,NaN,0,NaN,NaN
2,24357541,2177-02-10 14:00:00,659.416667,0,19004463,2176-12-16 18:11:00,2177-02-15 17:00:00,NaN,URGENT,TRANSFER FROM HOSPITAL,REHAB,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0,NaN,59.027559
3,22188993,2145-11-04 18:40:00,143.666667,0,11538389,2145-10-29 12:46:00,2145-11-09 17:30:00,2145-11-09 17:30:00,EW EMER.,EMERGENCY ROOM,DIED,Medicare,ENGLISH,MARRIED,UNKNOWN,2145-10-29 11:07:00,2145-10-29 14:00:00,1,-118.833333,74.007874
4,21880799,2134-05-22 17:58:00,54.966667,0,16606203,2134-05-20 09:51:00,2134-05-22 18:00:00,NaN,EW EMER.,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,?,MARRIED,HISPANIC/LATINO,2134-05-20 06:11:00,2134-05-20 11:15:00,0,NaN,66.070866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16399,29960248,2146-12-09 14:23:00,73.466667,1,16691924,2146-11-30 16:09:00,2146-12-17 17:04:00,NaN,DIRECT EMER.,CLINIC REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,WIDOWED,WHITE,NaN,NaN,0,NaN,61.011811
16400,29962016,2135-10-30 11:45:00,218.083333,1,16796371,2135-10-21 08:16:00,2135-11-04 13:30:00,NaN,EW EMER.,EMERGENCY ROOM,REHAB,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2135-10-21 05:47:00,2135-10-21 09:45:00,0,NaN,64.086614
16401,29974575,2131-03-03 19:54:00,98.900000,1,10020944,2131-02-27 15:34:00,2131-03-13 17:01:00,NaN,EW EMER.,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,NaN,UNKNOWN,2131-02-27 13:16:00,2131-02-27 16:40:00,0,NaN,70.039370
16402,29987115,2148-02-19 10:00:00,44.000000,1,19652570,2148-02-16 17:42:00,2148-02-20 19:00:00,2148-02-20 19:00:00,URGENT,TRANSFER FROM HOSPITAL,DIED,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,1,-33.000000,59.921260


In [14]:
mean_df.to_feather(mean_export_file)

In [15]:
#num_ventilations = last_value_pre['hadm_id'].value_counts()
#svdf = last_value_pre[last_value_pre['hadm_id'].isin(num_ventilations.index[num_ventilations ==2])]
#sns.countplot(svdf['valuenum'])

### Getting variability over last 6 hours

In [16]:
df_var = df.copy()
df_var = df[(df['timediffhour']>0)&(df['timediffhour']<6)]

varBP = df_var.groupby('hadm_id')['valuenum'].std()
varBP = varBP.reset_index()
varBP.columns = (stdColumns)

In [17]:
#plt.hist(merged_inner_var['timediffhour'])

In [18]:
std_df = pd.merge(left = svents, right=varBP, how = 'left', left_on='hadm_id', right_on='hadm_id')

In [19]:
std_df.to_feather(std_export_file)

new_df = pd.merge(left = svents, right=meanBP, how = 'left', left_on='hadm_id', right_on='hadm_id')
new_df

new_df = pd.merge(left = new_df, right=varBP, how = 'left', left_on='hadm_id', right_on='hadm_id')
new_df

new_df.drop(['temp','stdTemp'],axis=1,inplace=True)

new_df.to_csv('feature_table.csv',index= False)